In [1]:
import os
import re

In [2]:
# Создаём папку для выходных файлов. Если таковая уже имеется, то очищаем ее от старых файлов.
if os.path.exists("Output"):
    for file in os.listdir("Output/"):
        os.remove("Output/" + file)    
else:        
    os.mkdir("Output")

In [3]:
# Cчитаем количество строк, отведённых под тэги [header section]
def Skip_string (infile):
    count = 0
    with open (infile) as f:
        for line in f:
            if line.startswith("@"):
                count = count + 1
            else:
                break
    return count

In [8]:
def Make_tab(file, tags):
    k = 0
    list_input = [[]*50 for x in range(3)]
    with open (file) as f:
        for line in f:
            if k >= tags_num: 
                #print(line)
                col = line.split("\t")
                list_input[0].append(col[2])
                list_input[1].append(int(col[3]))
                list_input[2].append(col[5])         
            k +=1
    bam = list(zip(*list_input))
    bam = sorted(bam, key = lambda x: (x[0], x[1]))
    bam = list(zip(*bam))
    #print(bam)
    return(bam)
            

In [9]:
# Функция, получающая таблицу формата Bedgraph из листа, содержащего информацию о покрытии каждого элемента
# chr 10, [1 1 1 1 3 3 0 0 4 4 4] (здесь первый элемент имеет нулевой индекс)
#          _______ 
#              |            
#  >> [10 0 4 4]   ___
#                   |
#          [10 4 6 2]      ____
#                           |
#                  [10 8 11 3]

def Make_bedgraph(cover_list):
    
    flg = 0 # необходим для пропуска баз с нулевым покрытием, эти регионы не учитываются в bedgraph файле
    
    # отдельно считаем покрытие для первой базы из исходного файла 
    if cover_list[0]!= 0:
        bed_list[0].append(chrom)
        bed_list[1].append(0)
        bed_list[3].append(cover_list[0])
        flg = 1
        
    for i in range(1, len(cover_list)):
        #print(i)
        if cover_list[i] != cover_list[i-1]: 
            if flg!=0:
                bed_list[2].append(i)  # добавляем конец предыдущего полуинтервала покрытия = началу нового отрезка покрытия. 
            if cover_list[i]!= 0:  
                bed_list[0].append(chrom)
                bed_list[1].append(i)
                bed_list[3].append(cover_list[i])
                flg = 1
            if cover_list[i] == 0:
                flg = 0
                
    return(bed_list)   

In [10]:
in_dir = "Input/"
read_len = 300 # Initial length of reads. Default = 300

for file in os.listdir(in_dir): # Ведём рассчеты для всех файлов, помещенных в папку Input
    print(file)
    bed_list = [[]*50 for x in range(4)]
    tags_num = Skip_string(in_dir + file)
    bam = Make_tab(in_dir + file, tags_num)    
    
    chroms = list(set(bam[0])) # получаем список хромосом из .sam файла
    chroms.sort()
    print("Chromosomes: ", chroms)

    
    for chrom in chroms:
        
        print("Calculating chromosome {}...".format(chrom))
#         bam_for_chr = bam[bam['chr']== chrom]['coord']
#         bam_chr_indices = bam_for_chr.index.tolist()
        #print(bam_chr_indices)
    
        coverage_all = [0] * (max(bam[1]) + read_len) # cоздаем лист, который затем заполним информацией о покрытии каждого элемента
        for i in range(len(bam[0])):
            if bam[0][i] == chrom:
            
        
#         M alignment match (can be a sequence match or mismatch)
#         I insertion to the reference
#         D deletion from the reference
#         N skipped region from the reference
#         S soft clipping (clipped sequences present in SEQ)
#         H hard clipping (clipped sequences NOT present in SEQ)
#         P padding (silent deletion from padded reference)
           
                # Распознавание CIGAR операторов
                cigar = re.findall(r'(\d+)([A-Z]{1})', bam[2][i])   #cigar - лист вида: [('13', 'S'), ('138', 'M')]  
                # print(cigar)
                cursor = bam[1][i]

                for c in cigar:
                    if c[1] == "M" or c[1] == "=" or c[1] == "X":  # обрабатываем части ридов, которые заматчились
                        coord_start = cursor
                        coord_end = coord_start + int(c[0])
                        cursor = coord_end
                        for i in range(coord_start-1, coord_end-1):
                             coverage_all[i] += 1  
                    elif c[1] == "D" or c[1] == "N" or c[1] == "P":
                        cursor = cursor + int(c[0])   

        bed_list = Make_bedgraph(coverage_all)
        
    bed_df = list(zip(*bed_list))
    print(bed_df[:10])  #выводим первые 10 значений

    # Cохраняем файл 
    file = file.split(".")[0]
    f_out = open("Output/{}.bedgraph".format(file), "w")
    f_out.write("\n".join(['\t'.join(map(str, item)) for item in bed_df]))
            

for_test.sam
Chromosomes:  ['III', 'IV', 'V']
Calculating chromosome III...
Calculating chromosome IV...
Calculating chromosome V...
[('III', 2555, 2655, 1), ('III', 3100, 3200, 1), ('III', 6790, 6890, 1), ('III', 7234, 7334, 1), ('III', 21931, 22031, 1), ('III', 22351, 22451, 1), ('III', 25570, 25670, 1), ('III', 25996, 26096, 1), ('III', 27361, 27461, 1), ('III', 27756, 27856, 1)]
